<center>

# **PA5:  Unsupervised Learning**




# **Part 1 :Song Clustering with K‑Means(80 marks)**

In this part you will perform unsupervised clustering of
Spotify track features using the classic **K‑Means** algorithm.

You will:

1. Load and inspect the Spotify Features dataset.  
2. Drop purely metadata columns that should not influence clustering.  
3. Standardise numeric features (K‑Means is distance‑based).  
4. Use the **Elbow** and **Silhouette** methods to choose an appropriate *K*.  
5. Train a final K‑Means model and attach cluster labels.  
6. Visualise clusters in 2‑D via **PCA**.  

You are <span style="color: red;">not allowed</span> to use any libraries that have not been imported already


# **Step 1 : Import Libraries**

All of the necessary libraries for this part have been imported for you below. You may not use any other library apart from standard Python librares

In [ ]:
# You are not allowed import any additional libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

# **Step 2 : Load & Clean the Dataset** (5 marks)

Load the `SpotifyFeatures.csv` dataset into a DataFrame.

In this step, your goal is to:
- Load the dataset
- Check for any **null values**
- Fill or drop null values as appropriate


In [ ]:
# Step 2: Load the dataset over here

# Feature Selection Debate — What Should We Do with Categorical Columns? (5 marks)

We asked three of your TAs — **Abdullah**, **Ahsan**, and **Hamza** — how to handle **categorical features** in the Spotify dataset, such as:

- `genre` (e.g., "Pop", "Jazz", "Hip-Hop")  
- `key` (e.g., "C#", "D", "G")  
- `time_signature` (e.g., "3/4", "4/4")  
- `mode` (e.g., "Major", "Minor")

Here’s what each TA had to say:

---

### TA Opinions

- **Abdullah** says:  
  “We should remove all categorical columns. K-Means relies on numerical distance, using categories will distort the clustering.”

- **Ahsan** says:  
  “We should keep useful categorical features and label encode them.  
  For example: `Pop = 0`, `Jazz = 1`, `Hip-Hop = 2`. It’s compact and doesn’t add extra columns.”

- **Hamza** says:  
  “We should apply **one-hot encoding**, but only to **carefully chosen** categorical features:  
  - Keep those that are meaningful and have few categories (like `time_signature`)  
  - Avoid encoding features with too many unique values (like `genre`)  
---

### Your Task

- Whose approach do you agree with, and why?  
- Think critically about how categorical features might affect clustering results.

**Write your answer in the markdown cell below.**


## Answer Here 




# **Step 3: Drop All Categorical Features**

To keep this assignment focused and manageable, we’ll **remove all categorical features** for now.

That means you don’t need to worry about encoding or choosing which ones to keep. We’re doing this to simplify the clustering process and let you concentrate on the core idea behind K-Means.



In [ ]:
# Your code here

# **Step 4 : Standardize the Numerical Features** (10 marks)

Before standardizing the features, take a moment to think:

- Why do we actually need to standardize the features before applying K-Means clustering?  
- Don’t give a generic answer, instead, think about what the K-Means algorithm is actually doing when it forms clusters.

An example of a **generic answer to avoid**:
- We standardize so that all the features are on the same scale.”

That's not enough. Think deeper. What does the algorithm actually do? Why might raw numerical values affect that process?


📝 **Your Task**:  
Write your reasoning under the "Your Reasoning Heading" below.

## **Your Reasoning**:


---

Finally, you may proceed to **standardize the numerical features** in the code cell.

## **Standardization Instructions**

You must manually compute **z-scores** for each numeric feature without using `StandardScaler` or any machine learning library.



In [ ]:
# Your code here

# **Step 4: Implement K-Means as a Standalone Function** (20 marks)

Before you can evaluate different values of K or analyze cluster structure, you need to implement **K-Means** yourself — without using any ML libraries.

You will write a **standalone, reusable function** that can be used for:
- Computing WCSS for the Elbow method
- Getting cluster labels for the Silhouette score
- Running the final clustering model

---

## What Should Your K-Means Function Do?

Your function will take in:
- A 2D NumPy array `X` of standardized feature data
- A number of clusters `k`
- Optional parameters like number of iterations and convergence tolerance

It will output:
- Final cluster **centroids**
- Cluster **labels** (which cluster each point belongs to)
- The total **WCSS** (Within-Cluster Sum of Squares)

This will allow you to reuse the same logic for all the later steps of the assignment.

---

## What’s the Core Idea of K-Means?

Here’s the step-by-step logic:

1. **Initialize**:  
   - Randomly choose `k` points from the dataset to be the **initial centroids**.

2. **Assign Points to Clusters**:  
   - For each data point, compute its distance to each centroid.
   - Assign it to the nearest one.

3. **Update Centroids**:  
   - For each cluster, compute the **mean** of all points assigned to it.
   - This becomes the **new centroid**.

4. **Repeat**:  
   - Continue reassigning points and updating centroids until the centroids stop changing significantly, or a maximum number of iterations is reached.

---

## How Will You Use This Later?

Once you’ve implemented this function, you’ll call it repeatedly in future steps:
- To compute WCSS for multiple K values (for the Elbow method)
- To get cluster labels for computing Silhouette Scores
- To get the final clustering result when you pick your best K

This avoids writing the same logic over and over again.

---

In [ ]:
def kmeans(X, k, max_iters=100, tol=1e-4, verbose=False):
    """
    K-Means clustering from scratch.

    Parameters:
    - X: ndarray (n_samples, n_features), standardized data
    - k: number of clusters
    - max_iters: max number of iterations
    - tol: tolerance for centroid movement (L2 norm)
    - verbose: print internal info if True

    Returns:
    - centroids: final (k, n_features) array of cluster centers
    - labels: (n_samples,) array of assigned cluster indices
    - wcss: total within-cluster sum of squares
    """
    # Your code here

# **Step 5: Choose K with the Elbow (WCSS) Method** (15 marks)

Before we can run K-Means, we have to choose how many clusters (**K**) we want.  
But how do we know what a “good” K is?

---

## What’s WCSS?

WCSS stands for **Within-Cluster Sum of Squares**. It’s a number that tells us **how tight and compact our clusters are**.

Here’s the idea:

- Each point belongs to a cluster.
- That cluster has a center (called a **centroid**).
- For each point, we calculate how far it is from the center of its cluster.
- We square those distances and **add them all up** across all clusters.

This total is the WCSS.  
If your clusters are tight (all points close to the center), WCSS is low.  
If your clusters are messy and spread out, WCSS is high.

---

## What Is the Elbow Method?

Let’s say we try different values of K, like 2, 3, 4, all the way to 10 — and calculate the WCSS each time.

We then plot:
- X-axis = number of clusters (K)
- Y-axis = WCSS

Here’s what usually happens:
- At first, WCSS drops **a lot** as you increase K. That’s good — your clusters are getting tighter.
- But after a point, adding more clusters doesn’t help much — WCSS still drops, but only **a little**.

That turning point, where the drop starts to slow down, is called the **elbow**.

---

## Why Do We Pick the Elbow?

Let’s break it down:

- When you first increase K (e.g., from 2 to 3 to 4), K-Means is:
  - It’s discovering **natural groupings** in your data — separating points that belong in different clusters.
  - That’s why the WCSS (total error) drops **a lot** in the beginning — the clusters are becoming more meaningful and distinct.

But as you keep increasing K…

- At some point, the algorithm has already found the **main clusters** in the data.
- Any further increase in K just means:
  - You're splitting existing clusters into smaller sub-groups.
  - These new clusters don’t reveal new structure — they just divide things **within** already good clusters.
  - So the improvement in WCSS becomes **very small**.

That point — where we go from **discovering real structure** to just **splitting things for no good reason** — is what we call the **elbow**.

It’s where the WCSS curve changes from steep to flat.

So we pick the elbow as our K because:
- Before the elbow: we’re **finding real, meaningful clusters**.
- After the elbow: we’re just **overcomplicating** the model by breaking up groups that were already good.

---
## What About Random Initialization?

One small problem with K-Means:  
It starts with **random initial centroids** — and if it gets unlucky, it can give a **bad result**.

That’s why we don’t just run it once.

Instead, we run it **multiple times** (e.g., 5 random initializations), and keep the clustering with the **lowest WCSS**.

This makes the elbow curve more **reliable**, because:
- It avoids misleading results from a single unlucky run.
- It gives you the best possible WCSS value for each K.

So when you see a WCSS point on the elbow plot, know that it’s **not from one run** — it’s the best result from multiple trials with different random starting points.



## What You Need to Do

Now that you’ve implemented your `kmeans()` function, you will use it to apply the **Elbow Method** for identifying a suitable value of **K**.

Follow the steps below:

---

1. Run K-Means for different values of **K** ranging from **2 to 10**.

2. For each K, calculate the **WCSS** (Within-Cluster Sum of Squares) — this value is already returned by your `kmeans()` function.

3. **Important:**  
   Since K-Means starts with **random initial centroids**, it can produce different results on each run.  
   To get a more **reliable WCSS**, you must:
   - Run K-Means **5 times for each K**
   - Store the **lowest WCSS** among those 5 runs

   This avoids misleading results due to a single unlucky initialization.

4. Store the final (lowest) WCSS value for each K, and plot a line graph:
   - **X-axis** = values of K (2 through 10)  
   - **Y-axis** = corresponding WCSS values

5. **Visually inspect the plot** and look for the **"elbow"** — the point where WCSS stops decreasing rapidly.  
   This is the K where adding more clusters doesn’t significantly improve the model.


In [ ]:
def compute_wcss_range(X, k_range, n_init=5, verbose=True):
    """
    Computes WCSS for a range of cluster counts using the lowest WCSS out of multiple initializations.

    Parameters:
    - X: ndarray of shape (n_samples, n_features)
    - k_range: iterable of K values to evaluate (e.g., range(2, 11))
    - n_init: number of times to run KMeans per K
    - verbose: whether to print status

    Returns:
    - wcss_values: list of lowest WCSS per K
    """

    # Your code here

# Plot the curve here

# **Step 6: Validate K with the Silhouette Score** (15 marks)

Once you’ve used the Elbow Method to narrow down a possible K, it’s helpful to **validate** your choice using another metric: the **Silhouette Score**.

---

## What’s the Silhouette Score?

The **Silhouette Score** measures how well each point fits within its assigned cluster **compared to** other clusters.

It answers the question:
> “Is this point really in the right cluster, or would it fit better somewhere else?”

Here’s how it works:

For each data point:
- `a(i)` = average distance to other points in the **same cluster**
- `b(i)` = average distance to points in the **nearest other cluster**
- Silhouette score for that point =  
  **(b - a) / max(a, b)**

So:
- A score near **+1** → well-clustered
- A score near **0** → ambiguous or on a boundary
- A score **< 0** → possibly misclassified

---

## Why Use the Silhouette Score?

Unlike WCSS (which only measures **how compact** your clusters are), the Silhouette Score measures **how well-separated** and **internally consistent** they are.

It captures two ideas:
- **Cohesion**: Are the points close to others in their own cluster?
- **Separation**: Are they far from other clusters?

This makes it ideal for validating your K.

---

## What You Need to Do

1. Run your `kmeans()` function for each value of **K** from 2 to 10.
2. For each K, run K-Means **5 times** with different random initializations.
3. For each run, compute the **average silhouette score** using `compute_average_silhouette()`.
4. For each K, keep the **highest silhouette score** among the 5 runs.
5. Use `evaluate_silhouette_over_k()` to automate this process.
6. Plot the results using `plot_silhouette_scores()`.

---

## A little explanation of the functions

### `compute_average_silhouette(X, labels)`
- Computes the silhouette score **from scratch**.
- Input: dataset `X`, assigned cluster labels `labels`
- Output: average silhouette score (float)

### `evaluate_silhouette_over_k(X, k_range, n_init=5, sample_size=1000)`
- Tries multiple K values and multiple initializations.
- Uses `compute_average_silhouette()` internally.
- Returns a dictionary mapping each K to its best silhouette score.

### `plot_silhouette_scores(scores)`
- Takes the output of the previous function and draws a line plot.
- Helps you visually identify the **best K** (the one with the peak score).

---

## What You’re Looking For

- A clear **peak** in silhouette score across different values of K.
- The best K is typically the one with the **highest average silhouette score** — not too small (underfitting) and not too large (overfitting).



In [ ]:
def compute_average_silhouette(X, labels):
    """
    Computes the average silhouette score from scratch.

    Parameters:
    - X: ndarray of shape (n_samples, n_features)
    - labels: ndarray of shape (n_samples,) with cluster labels

    Returns:
    - mean silhouette score (float)
    """

    # Your code here

def evaluate_silhouette_over_k(X, k_range, n_init=5, sample_size=1000, random_state=None, verbose=True):
    """
    Evaluates silhouette scores for different values of K using custom KMeans and silhouette.

    Parameters:
    - X: ndarray (n_samples, n_features)
    - k_range: iterable of K values to evaluate
    - n_init: number of KMeans initializations per K
    - sample_size: max number of points to use for silhouette score
    - random_state: RNG seed
    - verbose: print progress

    Returns:
    - silhouette_scores: dict {K: best silhouette score}
    """

    # Your code here




def plot_silhouette_scores(scores):
    """
    Plots silhouette scores over different values of K.

    Parameters:
    - scores: dict {K: silhouette score}
    """

    # Your code here





# **Step 7 : Train Final K‑Means Model** (5 marks)

Choose the best K based on the elbow and silhouette plots.  
Then, train a final K-Means model using that K.
We fit the final model and attach the resulting cluster labels to `df`.

In [ ]:
# Your code here

best_k = 999  # Replace with the value you selected



# **Step 8: Visualize Clusters via PCA** (10 marks)

After training your final K-Means model, it’s important to **see what the clusters look like**.

But there’s a challenge: your dataset might have **many features** (dimensions), which humans can’t directly visualize.

---

## Why PCA?

We use **Principal Component Analysis (PCA)** to solve this.

PCA is a dimensionality reduction technique that transforms your data into a new coordinate system where:
- The **first axis (PC1)** captures the most variation in the data
- The **second axis (PC2)** captures the next most variation (orthogonal to the first)

This lets us compress the data into just **2 dimensions**, while keeping as much structure as possible.

---

## Why Visualize Clusters?

Once we project the data using PCA, we can:
- Plot each data point in this 2D space
- Color the points based on their **cluster labels**

This helps you:
- See how well-separated the clusters are
- Spot overlaps or outliers
- Get an intuitive sense of whether the clusters **make sense visually**

Even though PCA simplifies the data, **it often reveals structure** that's useful for understanding your clustering results.

---

## What You Need to Do

1. Use `PCA(n_components=2)` from `sklearn.decomposition` on your **standardized dataset**.
2. Transform the data and plot it in 2D using `matplotlib` or `seaborn`.
3. Use different colors for each cluster using the labels from your final K-Means model.

---



In [ ]:
# Your code here
# You can use the PCA class from sklearn.decomposition to reduce the dimensionality of the data

---

Now that you've visualized your clusters in 2D using PCA, take a moment to **analyze what you're seeing**:

- Are the clusters **visually separated**, or do they **overlap**?
- Does the shape and size of each cluster **look balanced**, or is one dominating the space?
- Can you spot any **outliers** or points that look like they could belong to more than one cluster?

> ✍️ **Your Task**:  
Write a short explanation (3–4 lines) interpreting the PCA plot above. Focus on:
- Whether the clusters are well-separated
- Whether this visualization supports your earlier decision for **K = 3**

Remember: PCA reduces dimensionality, so this is an approximation — but it’s still a **great sanity check** to confirm your clustering results.


 **Your Answer**
-


🎉 **Well done!**  
You have successfully completed PA5 Part 1

<hr>

**<h1><b> CS331: Denoising Autoencoders and Latent Space Visualization | <span style="color: #007BFF;"> Assignment 5 Section 2 [120 marks] </span></b></h1>**

<hr>




In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns
from torchsummary import summary
from sklearn.manifold import TSNE


if torch.backends.mps.is_available():
    device = torch.device('mps') # Apple Silicon GPU
elif torch.cuda.is_available():
    device = torch.device('cuda') # NVIDIA GPU
else:
    device = torch.device('cpu')

<hr>

**<h1><b><span style="color: #007BFF;"> Visualization Functions</span></b></h1>**

<hr>

Only use the following helper visualization functions  provided below

In [ ]:
# =========== Visualization Functions ==========

def displayImg(img, title=""):
    """
    Displays a tensor as an image. Handles both single images and grids of images.

    Args:
        img (torch.Tensor): The image tensor to display. Expected shape for a single image
                           is (C, H, W) or for a grid (C, H, W_total) created by
                           torchvision.utils.make_grid.
        title (str, optional): The title to display above the image. Defaults to "".

    Returns:
        None. Displays the image using matplotlib.
    """
    img = img.cpu()
    npimg = img.numpy()
    plt.figure(figsize=(8, 8))
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.title(title)
    plt.axis('off')
    plt.show()

def plot_comparison(original, noisy, reconstructed, n=5, title_prefix=""):
    """
    Displays a comparison grid of original, noisy, and reconstructed images.

    Args:
        original (torch.Tensor): A batch of original (clean) image tensors.
                                 Shape: (batch_size, C, H, W).
        noisy (torch.Tensor): A batch of corresponding noisy image tensors.
                              Shape: (batch_size, C, H, W).
        reconstructed (torch.Tensor): A batch of corresponding reconstructed image tensors
                                      output by the autoencoder. Shape: (batch_size, C, H, W).
        n (int, optional): The number of image triplets (original, noisy, reconstructed)
                           to display. Defaults to 5.
        title_prefix (str, optional): A string to prepend to the main title of the plot.
                                     Defaults to "".

    Returns:
        None. Displays the comparison plot using matplotlib.
    """
    original = original.cpu()
    noisy = noisy.cpu()
    reconstructed = reconstructed.cpu()

    plt.figure(figsize=(15, 5))
    for i in range(n):
        # --- Display original image ---
        ax = plt.subplot(3, n, i + 1)
        plt.imshow(original[i].squeeze(), cmap='gray')
        ax.set_title("Original")
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

        # --- Display noisy input image ---
        ax = plt.subplot(3, n, i + 1 + n)
        plt.imshow(noisy[i].squeeze(), cmap='gray')
        ax.set_title("Noisy Input")
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

        # --- Display reconstructed image ---
        ax = plt.subplot(3, n, i + 1 + 2 * n)
        plt.imshow(reconstructed[i].squeeze(), cmap='gray')
        ax.set_title("Reconstructed")
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

    plt.suptitle(f"{title_prefix} - Image Denoising Comparison", fontsize=16)
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()

<hr>

## <h1><b> Introduction to Denoising Autoencoders <span style="color: #007BFF;"></span></b></h1>

<hr>

### <h2><b> The Challenge of Noise in Data </b></h2>
<p>Imagine taking a photo in low light. The resulting image might appear grainy or have artifacts – this is noise. Noise is common in real-world data and can significantly degrade its quality. For machine learning models, noisy data can lead to poor performance in tasks like image classification, object detection, or medical image analysis, as the noise can obscure the underlying patterns the model needs to learn.</p>

### <h2><b> Autoencoders: Learning to Reconstruct </b></h2>
<p>An Autoencoder is a type of artificial neural network used for unsupervised learning, primarily aimed at learning efficient representations (encodings) of data. It consists of two parts:</p>
<ol>
    <li><b>Encoder:</b>     Compresses the input data into a lower-dimensional latent space (also called the bottleneck). This forces the encoder to capture the most salient features of the data.</li>
    <li><b>Decoder:</b>     Attempts to reconstruct the original input data from the compressed latent representation generated by the encoder.</li>
</ol>
<p>The autoencoder is trained by minimizing the difference between the original input and the reconstructed output (reconstruction error). Essentially, it tries to learn an identity function, but the bottleneck forces it to learn a useful, compressed representation along the way.</p>

### <h2><b> Denoising Autoencoders (DAEs): Learning Robust Representations </b></h2>
<p>A Denoising Autoencoder (DAE) is a specific type of autoencoder designed to tackle the problem of noise. Instead of learning to reconstruct the original input from itself, a DAE is trained to reconstruct the *original, clean* input from a *corrupted (noisy)* version of it.</p>
<ul>
    <li><b>Training Process:</b>
        <ol>
            <li>Take a clean data sample.</li>
            <li>Artificially add noise to create a corrupted version.</li>
            <li>Feed the <i>noisy</i> version into the DAE's encoder.</li>
            <li>The decoder generates a reconstruction.</li>
            <li>Calculate the reconstruction error between the decoder's output and the <i>original clean</i> data sample.</li>
            <li>Update the DAE's weights to minimize this error.</li>
        </ol>
    </li>
    <li><b>Why it Works:</b> By forcing the network to recover the underlying clean structure from noisy input, the DAE learns features that are robust to noise and capture the essential characteristics of the data distribution. It learns to separate the signal (the digit) from the noise.</li>
</ul>

<hr>

<h2><b> Linear vs. Convolutional Denoising Autoencoders </b></h2>
<p>In this assignment, we will implement and compare two architectures for DAEs:</p>
<ul>
    <li><b>Linear DAE:</b>  This architecture uses only fully connected (dense or `Linear`) layers. To process images, the input image is typically flattened into a long vector.
    <li><b>Convolutional DAE (CDAE):</b>    This architecture leverages convolutional layers (`Conv2d`) in the encoder and corresponding transposed convolutional layers (`ConvTranspose2d`, sometimes called deconvolutional layers) in the decoder.
</ul>

<hr>

<h2><b> MNIST Dataset for Denoising </b></h2>
<p>We will apply these DAEs to the <b>MNIST dataset</b>. MNIST is a standard benchmark dataset consisting of 70,000 images (60k training, 10k testing) of handwritten digits (0 through 9). Each image is a 28x28 pixel grayscale image. We will create noisy versions of these images by adding Gaussian noise, which will serve as the input to our DAEs, while the original clean images will be the target output for training.</p>

<hr>

<hr>

## <h1><b><span style="color: #007BFF;"> Hyperparameters</span></b></h1>

<hr>
<p>Hyperparameters are configuration settings used to control the learning process. They are set before training begins. Tuning these can significantly impact model performance.</p>

In [ ]:
BATCH_SIZE = 256
EPOCHS = 15   #dont change this
LEARNING_RATE = 0.001
NOISE_FACTOR = 0.5  #dont change this
LATENT_DIM = 64

<hr>

## <h1><b> Dataset Loading |<span style="color: #007BFF;"> MNIST (Clean & Noisy)</span></b></h1>

<hr>
<p>For training a Denoising Autoencoder, we require two parallel datasets:</p>
<ol>
    <li><b>Clean Dataset:</b> The original, noise-free MNIST images. These will serve as the <i>target</i> for our autoencoder's reconstruction during loss calculation.</li>
    <li><b>Noisy Dataset:</b> The MNIST images corrupted with artificial noise. These will serve as the <i>input</i> to our autoencoder during training and evaluation.</li>
</ol>
<p>We achieve this by defining two different transformation pipelines using `torchvision.transforms`.</p>

NOTE: Pre-written code is provided for you.

```python

In [ ]:
# ============== Data Transformations ==================
transform_clean = transforms.Compose([
    transforms.ToTensor(),
])

# --- Custom Transformation to add Gaussian Noise ---
class AddGaussianNoise(object):
    """
    Adds Gaussian noise to a tensor image.

    Args:
        mean (float): Mean of the Gaussian distribution used for noise. Default: 0.0.
        std (float): Standard deviation of the Gaussian distribution used for noise. Default: 1.0.
        noise_factor (float): A multiplier applied to the standard deviation to control the
                              intensity of the added noise. Default: 0.5.
    """
    def __init__(self, mean=0., std=1., noise_factor=0.5):
        self.std = std
        self.mean = mean
        self.noise_factor = noise_factor

    def __call__(self, tensor):
        """
        Applies the noise transformation.

        Args:
            tensor (torch.Tensor): Input image tensor (expected range [0.0, 1.0]).

        Returns:
            torch.Tensor: Image tensor with added Gaussian noise, clipped to [0.0, 1.0].
        """
        noise = torch.randn(tensor.size()) * self.std * self.noise_factor + self.mean
        noisy_tensor = tensor + noise
        return torch.clamp(noisy_tensor, 0., 1.)

    def __repr__(self):
        """String representation of the transformation."""
        return self.__class__.__name__ + f'(mean={self.mean}, std={self.std}, noise_factor={self.noise_factor})'

transform_noisy = transforms.Compose([
    transforms.ToTensor(),
    AddGaussianNoise(mean=0., std=1., noise_factor=NOISE_FACTOR)
])


# ================ Load Clean Data ======================

trainset_clean = torchvision.datasets.MNIST(root='./data',train=True,download=True,transform=transform_clean)
# DataLoader wraps the dataset, providing batching, shuffling, etc.
# shuffle=False is important here to ensure clean batches align with noisy batches.
trainloader_clean = torch.utils.data.DataLoader(trainset_clean,batch_size=BATCH_SIZE,shuffle=False)

testset_clean = torchvision.datasets.MNIST( root='./data',train=False,download=True,transform=transform_clean)
testloader_clean = torch.utils.data.DataLoader(testset_clean,batch_size=BATCH_SIZE,shuffle=False)

# ================ Load Noisy Data =====================

trainset_noisy = torchvision.datasets.MNIST(root='./data',train=True,download=True, transform=transform_noisy)
trainloader_noisy = torch.utils.data.DataLoader(trainset_noisy,batch_size=BATCH_SIZE,shuffle=False)

testset_noisy = torchvision.datasets.MNIST(root='./data',train=False,download=True,transform=transform_noisy)
testloader_noisy = torch.utils.data.DataLoader(testset_noisy,batch_size=BATCH_SIZE,shuffle=False)

<hr>

## <h1><b> Data Visualisation |<span style="color: #007BFF;"> Clean vs. Noisy MNIST</span></b></h1>

<hr>
<p>Before building the models, let's visualize some corresponding samples from the clean and noisy datasets. This helps confirm that the noise transformation is working as expected and gives an idea of the challenge the Denoising Autoencoder will face.</p>

In [ ]:
# --- Display the images ---
# Use displayImg to visualize the first 16 images from each batch

<hr>

## <h1 style="text-align: left;"><b>Task 1: Linear Denoising Autoencoder | <span style="color: #007BFF;">MNIST</span></b></h1>

<hr>
<p>Our first model is a Denoising Autoencoder constructed using only <strong>Linear (fully connected) layers</strong>. This architecture requires flattening the 2D input image into a 1D vector before feeding it into the encoder. The decoder then reconstructs the flattened vector, which represents the denoised image.

<strong>🚨 Important: When implementing the model, ensure that both the encoder and the decoder sections utilize a maximum of 3 `nn.Linear` layers each.🚨</strong></p>


### <h2 style="text-align: left;"> Model Definition | <span style="color: #007BFF;">Linear DAE</span></h2>
<p>We define the `LinearDenoisingAutoencoder` class. It contains an `encoder` sequence (Linear layers reducing dimensionality) and a `decoder` sequence (Linear layers increasing dimensionality back to the original). ReLU activations are used in hidden layers, and a Sigmoid activation is crucial for the final decoder layer to ensure the output pixel values are in the [0, 1] range.</p>

In [ ]:
class LinearDenoisingAutoencoder(nn.Module):
    def __init__(self):
        """
        Initializes the layers of the Linear Denoising Autoencoder.
        Defines the encoder and decoder sequential blocks.
        """
        super(LinearDenoisingAutoencoder, self).__init__()
        self.encoder = ...
        self.decoder = ...

    def forward(self, x):
        """
        Defines the forward pass of the autoencoder.

        Args:
            x (torch.Tensor): Input tensor representing a batch of images.
                              Expected shape: (batch_size, 1, 28, 28).

        Returns:
            torch.Tensor: Output tensor representing the reconstructed flattened images.
                          Shape: (batch_size, 784).
        """

        reconstruction = ...
        return reconstruction

### <h2 style="text-align: left;"> Trainer Class | <span style="color: #007BFF;">Linear DAE</span></h2>
<p>This `LinearDAETrainer` class encapsulates the logic for training, evaluating, and visualizing the `LinearDenoisingAutoencoder`. It handles:</p>
<ul>
    <li>Initialization of the model, optimizer, and loss function.</li>
    <li>The training loop (`train_epoch`), which processes batches of noisy input and calculates loss against clean targets.</li>
    <li>The evaluation loop (`evaluate`), which measures the reconstruction MSE on the test set without updating model weights.</li>
    <li>A main `train` method orchestrating the epochs and printing progress.</li>
    <li>Utility methods for plotting loss curves (`plot_loss`) and visualizing the denoising results (`visualize_denoising`).</li>
</ul>
<p>Crucially, the loss is always computed by comparing the model's output (reconstruction from noisy input) with the corresponding original clean image.</p>

In [ ]:
class LinearDAETrainer:
    def __init__(self, model, learning_rate=0.001, device=None):
        """
        Initializes the Trainer.

        Args:
            model (nn.Module): The LinearDenoisingAutoencoder model instance to train.
            learning_rate (float, optional): The learning rate for the Adam optimizer.
                                             Defaults to 0.001.
            device (torch.device, optional): The device (e.g., 'cuda', 'mps', 'cpu') to
                                             run the training on.
        """
        self.criterion = nn.MSELoss()
        self.optimizer = optim.Adam(self.model.parameters(), lr=self.lr)
        # Print a summary of the model architecture
        summary(self.model, input_size=(1, 28, 28), device=str(self.device))
  


    def train_epoch(self, noisy_loader, clean_loader):
        """
        Performs a single epoch of training.

        Iterates through the dataset, performs forward and backward passes,
        and updates model weights. Calculates loss against clean targets.

        Args:
            noisy_loader (DataLoader): DataLoader providing batches of noisy images (input).
            clean_loader (DataLoader): DataLoader providing corresponding batches of clean
                                       images (target).

        Returns:
            float: The average training loss for this epoch.
        """
        return 0

    def evaluate(self, noisy_loader, clean_loader):
        """
        Evaluates the model's performance on the test dataset.

        Calculates the reconstruction loss  between the model's output
        (from noisy input) and the clean target images without updating weights.

        Args:
            noisy_loader (DataLoader): DataLoader for the noisy test images (input).
            clean_loader (DataLoader): DataLoader for the clean test images (target).

        Returns:
            float: The average test loss for this epoch.
        """
        return 0

    def train(self, train_noisy_loader, train_clean_loader, test_noisy_loader, test_clean_loader, num_epochs):
        """
        Runs the complete training loop for a specified number of epochs.

        Calls train_epoch and evaluate for each epoch and prints the progress.

        Args:
            train_noisy_loader (DataLoader): DataLoader for noisy training data.
            train_clean_loader (DataLoader): DataLoader for clean training data.
            test_noisy_loader (DataLoader): DataLoader for noisy test data.
            test_clean_loader (DataLoader): DataLoader for clean test data.
            num_epochs (int): The total number of epochs to train for.

        Returns:
            None. Prints training progress.
        """


    def plot_loss(self):
        """
        Generates and displays a plot of training and test loss curves over epochs.
        """


    def visualize_denoising(self, noisy_loader, clean_loader, num_images=5):
        """
        Visualizes the denoising results by comparing original, noisy,
        and reconstructed images side-by-side.

        Args:
            noisy_loader (DataLoader): DataLoader for noisy test images.
            clean_loader (DataLoader): DataLoader for clean test images.
            num_images (int, optional): The number of image triplets to display.
                                        Defaults to 5.

        Returns:
            None. Displays the comparison plot.
        """
        # Use the dedicated plotting function plot_comparison

### <h2 style="text-align: left;">Model Training & Evaluation | <span style="color: #007BFF;">Linear DAE</span></h2>

- Instantiate the `LinearDenoisingAutoencoder` model.  
- Create a `LinearDAETrainer` object to manage training.  
- Start the training process
- After training:
  - Plot the loss curves to visualize the training progress.
  - Display qualitative results:
    - Show denoised images compared them with their corresponding noisy inputs and clean originals.

In [ ]:
# ================== Setup Linear DAE ==================


# ===================== Train Model =====================







In [ ]:
# ===================== Plot Loss =====================



In [ ]:
# ================= Visualize Results =================

<hr>

## <h1 style="text-align: left;"><b>Task 2: Convolutional Denoising Autoencoder | <span style="color: #007BFF;">MNIST</span></b></h1>

<hr>
<p>Next, we implement a more sophisticated Denoising Autoencoder using <strong>Convolutional layers</strong> (`Conv2d`) in the encoder and <strong>Transposed Convolutional layers</strong> (`ConvTranspose2d`) in the decoder. This architecture is inherently better suited for image data as it preserves and utilizes the spatial structure (pixel neighborhoods) through learnable filters.


<strong>🚨 Important: When implementing the model, ensure that both the encoder and the decoder sections utilize maximum 3 relevant convolutional (`nn.Conv2d`) or transposed convolutional (`nn.ConvTranspose2d`) layers each. 🚨</strong></p>



<p>The encoder will progressively reduce the spatial dimensions (height, width) while increasing the number of channels (features), and the decoder will reverse this process to reconstruct the clean image.</p>

<p>We explicitly separate the `encoder` and `decoder` components within the model definition. This allows us to easily extract the output of the encoder (the latent space representation) later for Task 3 (t-SNE visualization).</p>


In [ ]:
class ConvDenoisingAutoencoder(nn.Module):
    def __init__(self, latent_dim=LATENT_DIM):
        """
        Initializes the layers of the Convolutional Denoising Autoencoder.

        Args:
            latent_dim (int, optional): The desired dimensionality of the latent space
                                       (bottleneck). Defaults to LATENT_DIM hyperparameter.
        """
        super(ConvDenoisingAutoencoder, self).__init__()
        self.latent_dim = latent_dim 
        self.encoder = ...
        self.decoder = ...

    def forward(self, x):
        """
        Defines the forward pass of the convolutional autoencoder.

        Args:
            x (torch.Tensor): Input tensor representing a batch of images.
                              Expected shape: (batch_size, 1, 28, 28).

        Returns:
            tuple[torch.Tensor, torch.Tensor]: A tuple containing:
                - latent (torch.Tensor): The latent space representation. Shape: (batch_size, latent_dim).
                - reconstruction (torch.Tensor): The reconstructed image. Shape: (batch_size, 1, 28, 28).
        """
 
        latent = ...
        reconstruction = ...
        return latent, reconstruction

### <h2 style="text-align: left;"> Trainer Class | <span style="color: #007BFF;">Conv DAE</span></h2>
<p>The `ConvDAETrainer` class mirrors the structure of the `LinearDAETrainer` but is adapted for the `ConvDenoisingAutoencoder`. Key differences include:</p>
<ul>
    <li>Handling image-shaped inputs and outputs directly (no flattening needed for loss calculation).</li>
    <li>The `model.forward()` call returns both the latent vector and the reconstruction; only the reconstruction is used for calculating the denoising loss.</li>
    <li>Includes a crucial `get_latent_features` method. This method takes a DataLoader (typically containing *clean* images), passes them through the *encoder part* of the trained model, and collects the resulting latent vectors along with their true labels. This data is essential for the t-SNE visualization in Task 3.</li>
</ul>

In [ ]:
class ConvDAETrainer:
    """
    Manages the training, evaluation, visualization, and latent feature extraction
    for the ConvDenoisingAutoencoder model.
    """
    def __init__(self, model, learning_rate=0.001, device=None):
        """
        Initializes the Trainer.

        Args:
            model (nn.Module): The ConvDenoisingAutoencoder model instance.
            learning_rate (float, optional): Learning rate for the Adam optimizer. Defaults to 0.001.
            device (torch.device, optional): Device ('cuda', 'mps', 'cpu') for training.
                                             Auto-detects if None. Defaults to None.
        """
        self.criterion = nn.MSELoss()
        self.optimizer = optim.Adam(self.model.parameters(), lr=self.lr)
        # Display the model summary
        summary(self.model, input_size=(1, 28, 28), device=str(self.device))



    def train_epoch(self, noisy_loader, clean_loader):
        """
        Performs a single epoch of training for the Convolutional DAE.

        Args:
            noisy_loader (DataLoader): DataLoader for noisy input images.
            clean_loader (DataLoader): DataLoader for corresponding clean target images.

        Returns:
            float: Average training loss for the epoch.
        """
        return 0

    def evaluate(self, noisy_loader, clean_loader):
        """
        Evaluates the Convolutional DAE on the test dataset.

        Args:
            noisy_loader (DataLoader): DataLoader for noisy test images (input).
            clean_loader (DataLoader): DataLoader for clean test images (target).

        Returns:
            float: Average test loss for the epoch.
        """
        return 0
    
    def train(self, train_noisy_loader, train_clean_loader, test_noisy_loader, test_clean_loader, num_epochs):
        """
        Runs the full training and evaluation loop for the Conv DAE.

        Args:
            train_noisy_loader (DataLoader): DataLoader for noisy training data.
            train_clean_loader (DataLoader): DataLoader for clean training data.
            test_noisy_loader (DataLoader): DataLoader for noisy test data.
            test_clean_loader (DataLoader): DataLoader for clean test data.
            num_epochs (int): Total number of epochs to train.

        Returns:
            None. Prints training progress.
        """



    def plot_loss(self):
        """
        Generates and displays a plot of training and test loss curves over epochs.
        """
      

    def visualize_denoising(self, noisy_loader, clean_loader, num_images=5):
        """
        Visualizes the denoising performance by comparing original, noisy,
        and reconstructed images.

        Args:
            noisy_loader (DataLoader): DataLoader for noisy test images.
            clean_loader (DataLoader): DataLoader for clean test images.
            num_images (int, optional): Number of image triplets to display. Defaults to 5.

        Returns:
            None. Displays the comparison plot.
        """
        # Use the comparison plot_comparison function


    def get_latent_features(self, dataloader):
        """
        Extracts the latent space representations for all items in a given dataloader.
        This is typically used with the *clean* dataset after training to get latent
        vectors corresponding to the original digits for visualization (e.g., t-SNE).

        Args:
            dataloader (DataLoader): The DataLoader containing the data (usually clean images)
                                     for which to extract latent features.

        Returns:
            tuple[np.ndarray, np.ndarray]: A tuple containing:
                - latent_features (np.ndarray): A NumPy array of the extracted latent vectors.
                                                Shape: (num_samples, latent_dim).
                - all_labels (np.ndarray): A NumPy array of the corresponding true labels.
                                           Shape: (num_samples,).
        """

### <h2 style="text-align: left;"> Model Training & Evaluation | <span style="color: #007BFF;">Conv DAE</span></h2>
<p>Similar to Task 1, we now instantiate the `ConvDenoisingAutoencoder` and its `ConvDAETrainer`. We execute the training loop, plot the resulting loss curves, and visualize the denoising performance on sample test images. We expect the convolutional model to generally achieve lower reconstruction error and produce visually clearer reconstructions compared to the linear model.</p>

In [ ]:
# ================== Setup Conv DAE ==================

# ===================== Train Model =====================






In [ ]:
# ===================== Plot Loss =====================

In [ ]:
# ================= Visualize Results =================

<hr>

## <h1 style="text-align: left;"><b>Task 3: t-SNE Visualization of Latent Space | <span style="color: #007BFF;">Conv DAE</span></b></h1>

<hr>

### <h2><b> Diving Deeper: Visualizing What the Autoencoder Learned </b></h2>
<p>After training the Convolutional Denoising Autoencoder (which usually learns richer representations than the linear one), we want to understand the structure of the information captured in its compressed latent space. The encoder maps each input image (784 dimensions or 1x28x28) to a point in a lower-dimensional space (e.g., 64 dimensions defined by `LATENT_DIM`). While we can't directly "see" in 64 dimensions, we can use dimensionality reduction techniques to project these points into a 2D or 3D space that we *can* visualize.</p>

### <h2><b> Introducing t-SNE: A Tool for High-Dimensional Data Visualization </b></h2>
<p><strong>t-Distributed Stochastic Neighbor Embedding (t-SNE)</strong> is a state-of-the-art technique specifically designed for visualizing high-dimensional datasets in low dimensions (typically 2D or 3D). It's not a clustering algorithm itself, but it's excellent at revealing underlying cluster structures present in the data.</p>

<h4><b>Core Idea: Preserving Neighborhoods</b></h4>
<p>t-SNE works by modeling the similarity between high-dimensional data points and then trying to find a low-dimensional embedding where similar points remain close together and dissimilar points are pushed apart. It focuses on preserving the *local* structure of the data.</p>

1.  **Modeling High-Dimensional Similarities:**
    For every pair of high-dimensional points (our latent vectors), t-SNE computes a conditional probability
    $p_{j|i}$ that point $i$ would pick point $j$ as its neighbor if neighbors were picked in proportion to their probability density under a Gaussian centered at $i$.
    It then creates a joint probability:
    $$p_{ij} = \frac{p_{j|i} + p_{i|j}}{2n}$$
    representing the similarity between $i$ and $j$.

2.  **Modeling Low-Dimensional Similarities:**
    It defines a similar joint probability $q_{ij}$ for the corresponding points in the low-dimensional map (e.g., 2D).
    Crucially, it uses a heavy-tailed Student's t-distribution (with one degree of freedom) instead of a Gaussian here.
    This allows moderately dissimilar points in the high-dimensional space to be modeled further apart in the low-dimensional map, helping to prevent crowding and better separating clusters.

3.  **Minimizing Divergence:**
    t-SNE uses gradient descent to adjust the positions of the points in the low-dimensional map to minimize the Kullback-Leibler (KL) divergence between the two distributions of joint probabilities ($P$ in high-D and $Q$ in low-D).
    This optimization process effectively tries to make the low-dimensional map reflect the neighborhood structure of the high-dimensional data.

DW we wont be doing this manually and will use the library!    

<h4><b>Key Parameters (in `sklearn.manifold.TSNE`):</b></h4>
<ul>
    <li><b>`n_components` (int, default=2):</b> The target dimensionality (usually 2 for plotting).</li>
    <li><b>`perplexity` (float):</b> This parameter loosely relates to the number of nearest neighbors considered for each point. It influences the balance between local and global aspects. Values between 5 and 50 are common. <strong>Tuning perplexity can significantly change the visualization.</strong> Lower values focus more on very local structure, while higher values consider broader neighborhoods.</li>
    <li><b>`learning_rate` (float or 'auto', default='auto'):</b> Controls the step size during optimization. The default 'auto' setting (usually around 200) often works well. If the plot looks like a tight ball, the learning rate might be too low; if points seem randomly scattered, it might be too high.</li>
    <li><b>`n_iter` (int):</b> The number of optimization iterations. t-SNE requires enough iterations to converge. 250 is the minimum recommended, 1000 or more is typical.</li>
    <li><b>`init` (str or np.ndarray, default='random'):</b> Initialization method for the low-dimensional points. Using `'pca'` (`init='pca'`) often provides a more stable and globally consistent starting point than random initialization, potentially leading to better results and faster convergence.</li>
    <li><b>`random_state` (int or None, default=None):</b> Seed for the random number generator. Setting this ensures reproducibility of the t-SNE plot.</li>
    <li><b>`verbose` (int, default=0):</b> If > 0, prints progress messages during optimization. Useful for long runs.</li>
</ul>

<h4><b>How to Interpret the Plot:</b></h4>
<ul>
    <li><b>Clusters:</b> Look for distinct groups of points. In our case, we will color the points based on the true digit label (0-9). If the autoencoder learned meaningful features, we expect points corresponding to the same digit to form tight, relatively well-separated clusters.</li>
    <li><b>Cluster Separation:</b> The visual separation between clusters suggests how distinct the learned representations are for different classes.</li>
    <li><b>Caveats:</b>
        <ul>
            <li><strong>Distances between clusters are not always meaningful:</strong> t-SNE might place clusters far apart even if they were relatively close in the original space, or vice-versa. Focus on the grouping *within* clusters and the relative separation.</li>
            <li><strong>Cluster sizes are not meaningful:</strong> The area occupied by a cluster in the t-SNE plot doesn't directly reflect the variance or number of points in the original space.</li>
            <li>t-SNE is computationally intensive, especially for large datasets.</li>
        </ul>
    </li>
</ul>

<p>In this task, we will apply t-SNE to the latent vectors extracted from the <strong>Convolutional Denoising Autoencoder</strong> (using the clean test images as input to the encoder) and visualize the resulting 2D embedding.</p>

### <h2 style="text-align: left;"> Extracting Latent Features | <span style="color: #007BFF;">Conv DAE</span></h2>
<p>First, we need the latent space representations. We use the `get_latent_features` method from our `ConvDAETrainer`. It's important to pass the <strong>clean test set DataLoader</strong> (`testloader_clean`) to this method. Why? Because we want to visualize how the autoencoder represents the <i>original, underlying structure</i> of the digits in its latent space, not how it represents the noisy inputs.</p>

<p>Since t-SNE can be slow on large datasets (like the full 10,000 MNIST test images), you could use a random subset of 5000 data points for faster visualization.</p>

### <h2 style="text-align: left;"> Running t-SNE | <span style="color: #007BFF;">Conv DAE Latent Space</span></h2>
<p>Now we instantiate the `TSNE` object from `sklearn.manifold` with our chosen parameters (`n_components=2`, `perplexity`, `n_iter`, `init='pca'`, `random_state` for reproducibility, `verbose=1` to see progress). We then call the `fit_transform` method, passing in the extracted latent features. This performs the t-SNE optimization and returns the 2D coordinates for each latent vector.</p>
<p><i>Note: This step can take several minutes depending on the number of samples and the dimensionality of the latent space.</i></p>

In [ ]:
# ===================== Run t-SNE Algorithm ======================
# Initialize the t-SNE transformer object


# Apply t-SNE to the (subset of) latent features.
# ===========================================================

### <h2 style="text-align: left;"> Plotting t-SNE Results | <span style="color: #007BFF;">Conv DAE Latent Space</span></h2>
<p>Finally, we create a scatter plot of the 2D points generated by t-SNE. Each point represents an image from our test set (or subset). We use the `true_labels` (the actual digit, 0-9) to color-code the points. This allows us to visually inspect whether the latent space learned by the Convolutional DAE has successfully grouped images of the same digit together.</p>



In [ ]:
plt.figure(figsize=(12, 10)) # Create a reasonably large figure
# Create the scatter plot:

<hr>

## <h1><b> Theory Questions | <span style="color: #007BFF;">Assignment 5</span></b></h1>

<hr>

**Instructions:** Answer the following questions based on the concepts and implementations covered in this assignment. PLEASE BE BRIEF. No gpt taqreer. Actually yk what negative marking if you violate this rule

1.  **Core Autoencoder Concepts:**
    * a) What is the "bottleneck" or "latent space" in an autoencoder, and why is it important for learning useful representations?

    **Answer:**

2.  **Architectural Choices:**
    * a) What is the role of the `ConvTranspose2d` layers (Transposed Convolution) in the decoder of the Convolutional DAE?

     **Answer:**
     

3.  **Comparison and Analysis:**
    * a) Based on the visual results and the final test loss values, which model (Linear DAE or Convolutional DAE) performed better at the denoising task? Provide a reason for this difference(if there is any).

     **Answer:**


    * b) Can the encoder part of a trained DAE be used for other downstream tasks? If so, how and why might it be useful?

     **Answer:**

    * c) Do you notice anything special about the clusters of digits '4' and '9' in the t-SNE plot? If so why might this be?

    **Answer:**


Best of luck with your assignment! If you have any questions or need further assistance, feel free to ask. Happy coding!